In [1]:
import boto3
import traceback
import sys
import pandas as pd

In [2]:
aws_session = boto3.session.Session()
ts_query_client = aws_session.client('timestream-query')

In [3]:
def run_query(paginator, query_string):
    
        results = None
    
        try:
            page_iterator = paginator.paginate(QueryString=query_string)
            for page in page_iterator:
                results = parse_query_result(page)
                
                res_df = pd.DataFrame(results)
                
                return res_df
        except Exception as err:
            print("Exception while running query:", err)
            traceback.print_exc(file=sys.stderr)
            
            
def parse_query_result(query_result):
    query_status = query_result["QueryStatus"]
    column_info = query_result['ColumnInfo']
    
    results = []
    
    print(query_status)

    """
    progress_percentage = query_status["ProgressPercentage"]
    print(f"Query progress so far: {progress_percentage}%")

    bytes_scanned = float(query_status["CumulativeBytesScanned"]) / ONE_GB_IN_BYTES
    print(f"Data scanned so far: {bytes_scanned} GB")

    bytes_metered = float(query_status["CumulativeBytesMetered"]) / ONE_GB_IN_BYTES
    print(f"Data metered so far: {bytes_metered} GB")

    column_info = query_result['ColumnInfo']

    print("Metadata: %s" % column_info)
    print("Data: ")
    for row in query_result['Rows']:
        print(self._parse_row(column_info, row))
    """
    
    print("Metadata: %s" % column_info)
    print("Data: ")
    for row in query_result['Rows']:
        results.append(parse_row(column_info, row))
        #print(parse_row(column_info, row))
        
    return results
        

def parse_row(column_info, row):
    data = row['Data']
    row_output = []
    convert_data = {'BOOLEAN':bool, 'BIGINT':int, 'VARCHAR':str, 'DOUBLE':float}
    
    idComponent = None
    measure_name = ''
    measure_value = 0
    measure_time = None
    
    for j in range(len(data)):
        info = column_info[j]
        datum = data[j]
        #row_output.append(self._parse_datum(info, datum))
        #print(info)
        #print(datum)
        
        if datum.get('NullValue') != True:
            
            if info['Name'] == 'Component_Id':
                idComponent = int(datum['ScalarValue'])
            elif 'measure_value' in info['Name']:
                measure_value = convert_data[info['Type']['ScalarType']](datum['ScalarValue'])
            elif info['Name'] == 'measure_name':
                measure_name = str(datum['ScalarValue'])
            elif info['Name'] == 'time':
                measure_time = str(datum['ScalarValue'])
                
    return (idComponent, measure_name, measure_value, measure_time)

    #return "{%s}" % str(row_output)
    
def parse_datum(info, datum):
    return None
    

In [4]:
QUERY_1 = 'SELECT * FROM "octank-america-hvac"."thermafuser_readings" WHERE time between ago(5m) and now() ORDER BY time ASC '

In [5]:
paginator = ts_query_client.get_paginator('query')

res_df = run_query(paginator, QUERY_1)

{'ProgressPercentage': 100.0, 'CumulativeBytesScanned': 240540, 'CumulativeBytesMetered': 10000000}
Metadata: [{'Name': 'Factory_Id', 'Type': {'ScalarType': 'VARCHAR'}}, {'Name': 'Component_Id', 'Type': {'ScalarType': 'VARCHAR'}}, {'Name': 'Component_Type', 'Type': {'ScalarType': 'VARCHAR'}}, {'Name': 'Component_Name', 'Type': {'ScalarType': 'VARCHAR'}}, {'Name': 'measure_value::boolean', 'Type': {'ScalarType': 'BOOLEAN'}}, {'Name': 'measure_value::double', 'Type': {'ScalarType': 'DOUBLE'}}, {'Name': 'measure_name', 'Type': {'ScalarType': 'VARCHAR'}}, {'Name': 'time', 'Type': {'ScalarType': 'TIMESTAMP'}}]
Data: 


In [6]:
res_df = res_df.rename(columns={0:'id', 1:'measure', 2:'value', 3:'time'})
res_df.head()
res_df['time'] = pd.to_datetime(res_df['time'])
res_df.shape

(1995, 4)

In [7]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   id       1995 non-null   int64         
 1   measure  1995 non-null   object        
 2   value    1995 non-null   object        
 3   time     1995 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 62.5+ KB


In [8]:
id_df = res_df.loc[res_df['id'] == 65]
id_df

,id,measure,value,time
28,65,zoneTemperature,73.0,2021-03-22 05:11:50.373
29,65,roomOccupied,True,2021-03-22 05:11:50.373
30,65,occupiedHeatingSetpoint,68.07843,2021-03-22 05:11:50.373
31,65,supplyAir,58.099998,2021-03-22 05:11:50.373
32,65,occupiedCoolingSetpoint,73.07843,2021-03-22 05:11:50.373
...,...,...,...,...
1990,65,occupiedCoolingSetpoint,73.07843,2021-03-22 05:16:45.035
1991,65,zoneTemperature,71.599998,2021-03-22 05:16:45.035
1992,65,terminalLoad,36.0,2021-03-22 05:16:45.035
1993,65,roomOccupied,True,2021-03-22 05:16:45.035


In [9]:
pivoted_df = id_df.pivot(index="time", columns="measure", values="value")
pivoted_df.head()

measure,airflowFeedback,occupiedCoolingSetpoint,occupiedHeatingSetpoint,roomOccupied,supplyAir,terminalLoad,zoneTemperature
time,,,,,,,
2021-03-22 05:11:50.373,218.0,73.07843,68.07843,True,58.099998,70.0,73.0
2021-03-22 05:11:55.617,218.0,73.07843,68.07843,True,58.299999,70.0,73.099998
2021-03-22 05:12:00.870,218.0,73.07843,68.07843,True,58.400002,70.0,72.699997
2021-03-22 05:12:06.179,216.0,73.07843,68.07843,True,58.400002,69.0,72.699997
2021-03-22 05:12:11.433,214.0,73.07843,68.07843,True,58.599998,69.0,72.699997


In [10]:
day_quarters = {0:'0-5', 1:'6-11', 2:'12-17', 3:'18-23'}

#To ensure that all of the quarters are created
fake_entries = {'time':[None, None, None, None], 'airflowFeedback': [None, None, None, None], 
                'occupiedCoolingSetpoint':[None, None, None, None], 'roomOccupied':[None, None, None, None],
                'roomOccupied':[None, None, None, None], 'supplyAir':[None, None, None, None],
                'terminalLoad':[None, None, None, None], 'zoneTemperature':[None, None, None, None],
                'Day quarter': [0, 1, 2, 3]
               }
quarters_df = pd.DataFrame(fake_entries)
quarters_df.head()


,time,airflowFeedback,occupiedCoolingSetpoint,roomOccupied,supplyAir,terminalLoad,zoneTemperature,Day quarter
0,None,None,None,None,None,None,None,0
1,None,None,None,None,None,None,None,1
2,None,None,None,None,None,None,None,2
3,None,None,None,None,None,None,None,3


In [11]:
pivoted_df = pivoted_df.reset_index()

pivoted_df['Day quarter'] = pivoted_df['time'].map(lambda x: x.hour//6)
concat_df = pd.concat([pivoted_df, quarters_df], axis=0)

In [12]:
concat_df.shape

(61, 9)

In [13]:
dummies = pd.get_dummies(concat_df['Day quarter'])
concat_df = pd.concat([concat_df, dummies], axis=1)
concat_df.rename(columns=day_quarters, inplace=True)

In [14]:
concat_df

,time,airflowFeedback,occupiedCoolingSetpoint,occupiedHeatingSetpoint,roomOccupied,supplyAir,terminalLoad,zoneTemperature,Day quarter,0-5,6-11,12-17,18-23
0,2021-03-22 05:11:50.373,218.0,73.07843,68.07843,True,58.099998,70.0,73.0,0,1,0,0,0
1,2021-03-22 05:11:55.617,218.0,73.07843,68.07843,True,58.299999,70.0,73.099998,0,1,0,0,0
2,2021-03-22 05:12:00.870,218.0,73.07843,68.07843,True,58.400002,70.0,72.699997,0,1,0,0,0
3,2021-03-22 05:12:06.179,216.0,73.07843,68.07843,True,58.400002,69.0,72.699997,0,1,0,0,0
4,2021-03-22 05:12:11.433,214.0,73.07843,68.07843,True,58.599998,69.0,72.699997,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2021-03-22 05:16:45.035,112.0,73.07843,68.07843,True,59.700001,36.0,71.599998,0,1,0,0,0
0,NaT,None,None,NaN,None,None,None,None,0,1,0,0,0
1,NaT,None,None,NaN,None,None,None,None,1,0,1,0,0
2,NaT,None,None,NaN,None,None,None,None,2,0,0,1,0


In [15]:
concat_df = concat_df.dropna(axis=0, subset=['time'])
concat_df['airflowRoll'] = concat_df['airflowFeedback'].rolling(window=12).mean()
concat_df['supplyAirRoll'] = concat_df['supplyAir'].rolling(window=12).mean()
concat_df['zoneTemperatureRoll'] = concat_df['zoneTemperature'].rolling(window=12).mean()

<ipython-input-15-e2cce588579c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat_df['airflowRoll'] = concat_df['airflowFeedback'].rolling(window=12).mean()
<ipython-input-15-e2cce588579c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat_df['supplyAirRoll'] = concat_df['supplyAir'].rolling(window=12).mean()
<ipython-input-15-e2cce588579c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [16]:
concat_df

,time,airflowFeedback,occupiedCoolingSetpoint,occupiedHeatingSetpoint,roomOccupied,supplyAir,terminalLoad,zoneTemperature,Day quarter,0-5,6-11,12-17,18-23,airflowRoll,supplyAirRoll,zoneTemperatureRoll
0,2021-03-22 05:11:50.373,218.0,73.07843,68.07843,True,58.099998,70.0,73.0,0,1,0,0,0,NaN,NaN,NaN
1,2021-03-22 05:11:55.617,218.0,73.07843,68.07843,True,58.299999,70.0,73.099998,0,1,0,0,0,NaN,NaN,NaN
2,2021-03-22 05:12:00.870,218.0,73.07843,68.07843,True,58.400002,70.0,72.699997,0,1,0,0,0,NaN,NaN,NaN
3,2021-03-22 05:12:06.179,216.0,73.07843,68.07843,True,58.400002,69.0,72.699997,0,1,0,0,0,NaN,NaN,NaN
4,2021-03-22 05:12:11.433,214.0,73.07843,68.07843,True,58.599998,69.0,72.699997,0,1,0,0,0,NaN,NaN,NaN
5,2021-03-22 05:12:16.676,209.0,73.07843,68.07843,True,58.599998,67.0,72.699997,0,1,0,0,0,NaN,NaN,NaN
6,2021-03-22 05:12:21.936,207.0,73.07843,68.07843,True,58.400002,67.0,72.599998,0,1,0,0,0,NaN,NaN,NaN
7,2021-03-22 05:12:27.194,173.0,73.07843,68.07843,True,58.900002,56.0,71.900002,0,1,0,0,0,NaN,NaN,NaN
8,2021-03-22 05:12:32.466,150.0,73.07843,68.07843,True,58.900002,48.0,71.599998,0,1,0,0,0,NaN,NaN,NaN
9,2021-03-22 05:12:37.722,154.0,73.07843,68.07843,True,59.200001,50.0,71.699997,0,1,0,0,0,NaN,NaN,NaN


In [17]:
unique_ids = res_df['id'].unique()

dfs = {}

for identifier in unique_ids:
    
    id_df = res_df.loc[res_df['id'] == identifier]
    pivoted_df = id_df.pivot(index="time", columns="measure", values="value")
    
    dfs[identifier] = pivoted_df

    #print(id_df)
    #print(pivoted_df)


#pivoted = res_df.pivot(index="time", columns="measure", values="value")
#pivoted.head()

In [ ]:
json_df = dfs[65].to_json()
dfs[65].head()

In [ ]:
recovered_df = pd.read_json(json_df)
recovered_df.head()

In [ ]:
for key in dfs.keys():
    
    rolled = dfs[key].rolling
    
    print('Identifier: ' + str(key))
    print(dfs[key].head())
    print('\n\n')